In [12]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
#Models to use
from sklearn.linear_model import LogisticRegression

# Data

In [85]:

toy_educ = ['HS-grad', 'Doctorate', 'Masters', 'Bachelors', 'Doctorate'] * 2
toy_mar = ['widowed', 'divorced', 'married', 'single', 'married'] * 2 
toy_sex = ['Male', 'Male', 'Male', 'Female', 'Female'] * 2
toy_inc = [1, 0, 1, 0, 1] * 2
toy_educ_num = [3,4,7,2, 3,2,1,5,2,2]
toy_data = pd.DataFrame({'education':toy_educ , 'marital.status':toy_mar, 'sex':toy_sex, 'income':toy_inc, 'education.num':toy_educ_num})
toy_data


,education,marital.status,sex,income,education.num
0,HS-grad,widowed,Male,1,3
1,Doctorate,divorced,Male,0,4
2,Masters,married,Male,1,7
3,Bachelors,single,Female,0,2
4,Doctorate,married,Female,1,3
5,HS-grad,widowed,Male,1,2
6,Doctorate,divorced,Male,0,1
7,Masters,married,Male,1,5
8,Bachelors,single,Female,0,2
9,Doctorate,married,Female,1,2


In [86]:
RAW_DATA = toy_data
FEAT_OF_INT = ['education', 'marital.status', 'sex', 'income'] #The features we are interested in

## Transform categorical values

In [105]:
from sklearn.preprocessing import OneHotEncoder
cat_feats = list(RAW_DATA.select_dtypes("object").keys())  #names of all categorical features
# cat_feats.remove('income') #keep these values as is 
print(f"{len(cat_feats)} categorical features found: {cat_feats}")

cat_feat_encoder = OneHotEncoder(sparse_output=False, drop='if_binary').set_output(transform="pandas")
cat_feats_encoded = cat_feat_encoder.fit_transform(RAW_DATA[cat_feats])
CAT_FEAT_OF_INT = cat_feat_encoder.get_feature_names_out()
cat_feats_encoded

3 categorical features found: ['education', 'marital.status', 'sex']


,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,marital.status_divorced,marital.status_married,marital.status_single,marital.status_widowed,sex_Male
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
6,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [106]:
CAT_FEAT_OF_INT

array(['education_Bachelors', 'education_Doctorate', 'education_HS-grad',
       'education_Masters', 'marital.status_divorced',
       'marital.status_married', 'marital.status_single',
       'marital.status_widowed', 'sex_Male'], dtype=object)

In [109]:
cat_feat_encoder.inverse_transform(cat_feats_encoded)

array([['HS-grad', 'widowed', 'Male'],
       ['Doctorate', 'divorced', 'Male'],
       ['Masters', 'married', 'Male'],
       ['Bachelors', 'single', 'Female'],
       ['Doctorate', 'married', 'Female'],
       ['HS-grad', 'widowed', 'Male'],
       ['Doctorate', 'divorced', 'Male'],
       ['Masters', 'married', 'Male'],
       ['Bachelors', 'single', 'Female'],
       ['Doctorate', 'married', 'Female']], dtype=object)

In [89]:
RAW_DATA

,education,marital.status,sex,income,education.num
0,HS-grad,widowed,Male,1,3
1,Doctorate,divorced,Male,0,4
2,Masters,married,Male,1,7
3,Bachelors,single,Female,0,2
4,Doctorate,married,Female,1,3
5,HS-grad,widowed,Male,1,2
6,Doctorate,divorced,Male,0,1
7,Masters,married,Male,1,5
8,Bachelors,single,Female,0,2
9,Doctorate,married,Female,1,2


In [92]:
DATA = pd.concat([RAW_DATA.drop(columns=cat_feats), cat_feats_encoded], axis=1)
DATA

,income,education.num,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,marital.status_divorced,marital.status_married,marital.status_single,marital.status_widowed,sex_Male
0,1,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,1,7,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
3,0,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
5,1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
6,0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
7,1,5,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,0,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


## Split train and test

In [93]:
train, test = train_test_split(DATA, test_size=0.3)
print(f"Train size: {len(train)}, test size: {len(test)}")

Train size: 7, test size: 3


In [94]:
train

,income,education.num,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,marital.status_divorced,marital.status_married,marital.status_single,marital.status_widowed,sex_Male
4,1,3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,7,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
6,0,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
0,1,3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
7,1,5,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
8,0,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [95]:
x_train = train.drop(columns=['income'])
y_train = train['income']

x_test  = test.drop(columns=['income'])
y_test  = test['income']

# Logistic regression

In [96]:
# Your code
logreg = LogisticRegression().fit(x_train, y_train)

y_pred= list(logreg.predict(x_test))

logreg_results = test.copy() #get all the data of the test set
logreg_results = logreg_results.filter(items=FEAT_OF_INT) #keep only relevant info
logreg_results['y_pred'] = y_pred

In [99]:
FEAT_OF_INT

['education', 'marital.status', 'sex', 'income']

In [100]:
TEST_COPY = test.copy()
TEST_COPY

,income,education.num,education_Bachelors,education_Doctorate,education_HS-grad,education_Masters,marital.status_divorced,marital.status_married,marital.status_single,marital.status_widowed,sex_Male
5,1,2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,0,4,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0,2,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [101]:
TEST_COPY.filter(FEAT_OF_INT)

,income
5,1
1,0
3,0


In [97]:
logreg_results

,income,y_pred
5,1,1
1,0,1
3,0,0
